In [1]:
!pip install statsforecast

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from statsforecast.models import (AutoARIMA, HoltWinters)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import OneHotEncoder
from pylab import rcParams
import os
  
# Ignore harmless warnings 
import warnings 
warnings.filterwarnings("ignore") 


StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 57.8 MB/s eta 0:00:00


In [2]:
# Load dataset
konya = pd.read_parquet("abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/konya_features.parquet")

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 4, Finished, Available, Finished)

In [3]:
# vizualize dataset
konya.head()
konya.tail()

# print info
print(konya.info())

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 5, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532614 entries, 0 to 532613
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   store_id               532614 non-null  object        
 1   product_id             532614 non-null  object        
 2   date                   532614 non-null  datetime64[ns]
 3   sales                  532614 non-null  float64       
 4   revenue                532614 non-null  float64       
 5   stock                  532614 non-null  float64       
 6   price                  531602 non-null  float64       
 7   promo_type_1           532614 non-null  category      
 8   promo_bin_1            75780 non-null   category      
 9   promo_type_2           532614 non-null  category      
 10  promo_bin_2            479 non-null     category      
 11  promo_discount_2       479 non-null     float64       
 12  promo_discount_type_2  479 non-null     cate

In [4]:
# change dtypes 

cat_col = ['store_id', 'product_id', 'season', 'week']
konya[cat_col] = konya[cat_col].astype('category')

# Drop columns not necessary
df = konya.copy()
# List of columns to drop
columns_to_drop = [
    'promo_bin_1', 
    'promo_bin_2',
    'revenue', 
    'promo_discount_2', 
    'promo_discount_type_2', 
    'hierarchy2_id', 
    'hierarchy3_id', 
    'hierarchy4_id', 
    'hierarchy5_id', 
    'city_id_old', 
    'country_id', 
    'city_code',
    'product_length', 
    'product_depth',
    'product_width',
    'weekday'
]

# Drop the specified columns
df = df.drop(columns=columns_to_drop, errors='ignore')

#confirm drop
df.info()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 6, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532614 entries, 0 to 532613
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   store_id       532614 non-null  category      
 1   product_id     532614 non-null  category      
 2   date           532614 non-null  datetime64[ns]
 3   sales          532614 non-null  float64       
 4   stock          532614 non-null  float64       
 5   price          531602 non-null  float64       
 6   promo_type_1   532614 non-null  category      
 7   promo_type_2   532614 non-null  category      
 8   cluster_id     532614 non-null  category      
 9   hierarchy1_id  532614 non-null  category      
 10  storetype_id   532614 non-null  category      
 11  store_size     532614 non-null  int64         
 12  season         532614 non-null  category      
 13  week           532614 non-null  category      
 14  holiday        532614 non-null  category      
 15  

In [5]:
# Filter products that are present in 2019
products_in_2019 = df[df['date'].dt.year == 2019]['product_id'].unique()

# Filter the original DataFrame to include only products sold in 2019
df_filtered = df[df['product_id'].isin(products_in_2019)]

# Display the last few rows of the filtered DataFrame
df_filtered.tail()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 7, Finished, Available, Finished)

,store_id,product_id,date,sales,stock,price,promo_type_1,promo_type_2,cluster_id,hierarchy1_id,storetype_id,store_size,season,week,holiday,month_name
532609,S0142,P0733,2019-09-30,0.0,12.0,0.75,PR14,PR03,cluster_7,H00,ST04,31,3,40,N,Sep
532610,S0142,P0741,2019-09-30,0.0,3.0,32.90,PR10,PR03,cluster_0,H01,ST04,31,3,40,N,Sep
532611,S0142,P0742,2019-09-30,0.0,5.0,69.90,PR07,PR03,cluster_0,H01,ST04,31,3,40,N,Sep
532612,S0142,P0747,2019-09-30,0.0,16.0,21.90,PR14,PR03,cluster_0,H01,ST04,31,3,40,N,Sep
532613,S0142,P0748,2019-09-30,0.0,18.0,18.90,PR14,PR03,cluster_0,H01,ST04,31,3,40,N,Sep


In [6]:
# Filter rows where 'unique_id' is 'P0035' that only appears in 2018 to confirm filtering
filtered_df = df_filtered[df_filtered['product_id'] == 'P0531']

print(filtered_df.tail())

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 8, Finished, Available, Finished)

       store_id product_id       date  sales  stock  price promo_type_1  \
452911    S0142      P0531 2018-09-20    0.0   14.0   1.15         PR14   
453135    S0142      P0531 2018-09-21    8.0    6.0   1.15         PR14   
453359    S0142      P0531 2018-09-22    0.0    6.0   1.15         PR14   
453583    S0142      P0531 2018-09-23    1.0    5.0   1.15         PR14   
453807    S0142      P0531 2018-09-24    6.0    0.0   1.15         PR14   

       promo_type_2 cluster_id hierarchy1_id storetype_id  store_size season  \
452911         PR03  cluster_9           H00         ST04          31      3   
453135         PR03  cluster_9           H00         ST04          31      3   
453359         PR03  cluster_9           H00         ST04          31      3   
453583         PR03  cluster_9           H00         ST04          31      3   
453807         PR03  cluster_9           H00         ST04          31      3   

       week holiday month_name  
452911   38       N        Sep  
45

In [7]:
# Get unique store_ids
unique_store_ids = df_filtered['store_id'].unique()

# Create a dictionary of DataFrames, each keyed by store_id
store_datasets = {}
for store_id in unique_store_ids:
    store_datasets[store_id] = df_filtered[df_filtered['store_id'] == store_id].copy()

# dataset for store_id 'S0142'
store_S0142	 = store_datasets['S0142']
print("Store S0142 dataset:")
print(store_S0142.head())

# dataset for store_id S0094
store_S0094	 = store_datasets['S0094']
print("Store S0094 dataset:")
print(store_S0094.head())

# dataset for store_id S0030
store_S0030	 = store_datasets['S0030']
print("Store S0030 dataset:")
print(store_S0030.head())


StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 9, Finished, Available, Finished)

Store S0142 dataset:
       store_id product_id       date  sales  stock  price promo_type_1  \
329162    S0142      P0005 2017-01-02    0.0    2.0  33.90         PR14   
329163    S0142      P0015 2017-01-02    0.0   12.0   2.60         PR14   
329164    S0142      P0017 2017-01-02    0.0   17.0   1.49         PR14   
329165    S0142      P0018 2017-01-02    0.0    2.0   1.95         PR14   
329168    S0142      P0051 2017-01-02    0.0   15.0   0.70         PR14   

       promo_type_2 cluster_id hierarchy1_id storetype_id  store_size season  \
329162         PR03  cluster_9           H03         ST04          31      1   
329163         PR03  cluster_1           H00         ST04          31      1   
329164         PR03  cluster_6           H00         ST04          31      1   
329165         PR03  cluster_4           H00         ST04          31      1   
329168         PR03  cluster_7           H00         ST04          31      1   

       week holiday month_name  
329162    1   

In [8]:
# select store as df
df = store_S0030.copy() # replace according to store


StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 10, Finished, Available, Finished)

In [9]:
# get number of cluster_ids from store

store_S0030['cluster_id'].nunique()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 11, Finished, Available, Finished)

10

## S0030

In [10]:
# Change the granularity of time series data from daily to weekly

# Set 'ds' column as the index
df.set_index('date', inplace=True)
# Sort the DataFrame by the index (dates)
df.sort_index(inplace=True)

# Group by 'product_id' and resample to weekly frequency
df_weekly = df.groupby(['cluster_id']).resample('W-MON').agg({
    'sales': 'sum',  # Aggregate sales data
    'stock': 'max',  # Take the max stock value of the week
    'price': 'mean',  # Last price for the week
    'promo_type_1': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'promo_type_2': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    #'cluster_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'hierarchy1_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'storetype_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'store_size': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'season': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'holiday': lambda x: x.mode().iloc[0] if not x.mode().empty else None, 
    'week': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'month_name': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
}).reset_index()

# Display the weekly aggregated DataFrame
print(df_weekly.head())

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 12, Finished, Available, Finished)

  cluster_id       date  sales  stock     price promo_type_1 promo_type_2  \
0  cluster_0 2017-01-02    5.0   41.0  7.148214         PR14         PR03   
1  cluster_0 2017-01-09   28.0   41.0  7.528750         PR14         PR03   
2  cluster_0 2017-01-16   49.0   46.0  7.987438         PR14         PR03   
3  cluster_0 2017-01-23   43.0   43.0  8.065347         PR14         PR03   
4  cluster_0 2017-01-30   45.0   74.0  7.888500         PR14         PR03   

  hierarchy1_id storetype_id  store_size  season holiday  week month_name  
0           H00         ST03        13.0     1.0       N   1.0        Jan  
1           H00         ST03        13.0     1.0       N   2.0        Jan  
2           H00         ST03        13.0     1.0       N   3.0        Jan  
3           H00         ST03        13.0     1.0       N   4.0        Jan  
4           H00         ST03        13.0     1.0       N   5.0        Jan  


In [11]:
df_weekly.info()
#df.info()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 13, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   cluster_id     1440 non-null   category      
 1   date           1440 non-null   datetime64[ns]
 2   sales          1440 non-null   float64       
 3   stock          1423 non-null   float64       
 4   price          1423 non-null   float64       
 5   promo_type_1   1423 non-null   object        
 6   promo_type_2   1423 non-null   object        
 7   hierarchy1_id  1423 non-null   object        
 8   storetype_id   1423 non-null   object        
 9   store_size     1423 non-null   float64       
 10  season         1423 non-null   float64       
 11  holiday        1423 non-null   object        
 12  week           1423 non-null   float64       
 13  month_name     1423 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(6), object(6)
memory usage: 1

In [12]:
# Sort DataFrame by 'date'
df_weekly = df_weekly.sort_values(by='date')
#df = df.sort_values(by='date')

# Reset index after sorting
df_weekly.reset_index(drop=True, inplace=True)
#df.reset_index(drop=True, inplace=True)

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 14, Finished, Available, Finished)

In [13]:
df_weekly.tail()
#df.tail()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 15, Finished, Available, Finished)

,cluster_id,date,sales,stock,price,promo_type_1,promo_type_2,hierarchy1_id,storetype_id,store_size,season,holiday,week,month_name
1435,cluster_2,2019-09-30,0.0,21.0,13.075000,PR14,PR03,H00,ST03,13.0,3.0,N,39.0,Sep
1436,cluster_1,2019-09-30,1.0,11.0,5.935714,PR14,PR03,H00,ST03,13.0,3.0,N,39.0,Sep
1437,cluster_0,2019-09-30,31.0,26.0,20.314433,PR14,PR03,H00,ST03,13.0,3.0,N,39.0,Sep
1438,cluster_8,2019-09-30,9.0,70.0,3.496429,PR14,PR03,H00,ST03,13.0,3.0,N,39.0,Sep
1439,cluster_9,2019-09-30,21.0,49.0,9.550000,PR14,PR03,H00,ST03,13.0,3.0,N,39.0,Sep


# **ARIMA**

### All time range included

In [14]:
# Adjust column names if needed
df_weekly = df_weekly.rename(columns={
    'cluster_id': 'unique_id',
    'date': 'ds',
    'sales': 'y'
})

#funcao do arima
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()


# Rename variable
data = df_weekly.copy()

df_product = data.copy()

# Ensure 'unique_id' and 'ds' are properly formatted
df_product['unique_id'] = df_product['unique_id'].astype(str)
df_product['ds'] = pd.to_datetime(df_product['ds'])

# Separate sales and date from exogenous variables
data2 = df_product[['ds', 'unique_id', 'y']]

# Split data in train for different time-range
train = data2.loc[data2['ds'] < '2019-10-01']
# Count the occurrences of each product
product_counts = train['unique_id'].value_counts()
# Filter products that appear at least 5 times (to not arise error in crossvalidation)
products_to_keep = product_counts[product_counts >= 5].index
train = train[train['unique_id'].isin(products_to_keep)]



# Define AutoArima model
model = [AutoARIMA(season_length=52)]
#Forecast
sf = StatsForecast(models=model, freq='W-MON', n_jobs=-1)

#Cross-validation
crossvalidation_df = sf.cross_validation(
    df = train,
    h = 2,
    step_size = 2,
    n_windows = 3, 
    level = [90]
  )

print("crossvalidation_df:")
print(crossvalidation_df.head())
p = crossvalidation_df 
p = p.reset_index()
p.rename(columns={'index': 'unique_id'}, inplace=True)


print (p.head())

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 16, Finished, Available, Finished)

crossvalidation_df:
                  ds     cutoff     y  AutoARIMA  AutoARIMA-lo-90  \
unique_id                                                           
cluster_0 2019-08-26 2019-08-19  63.0  31.151754         9.746480   
cluster_0 2019-09-02 2019-08-19  15.0  31.888296         9.671510   
cluster_0 2019-09-09 2019-09-02  35.0  35.477863        13.414785   
cluster_0 2019-09-16 2019-09-02  40.0  31.131956         8.417244   
cluster_0 2019-09-23 2019-09-16  42.0  36.065189        13.810037   

           AutoARIMA-hi-90  
unique_id                   
cluster_0        52.557030  
cluster_0        54.105083  
cluster_0        57.540939  
cluster_0        53.846668  
cluster_0        58.320343  
   unique_id         ds     cutoff     y  AutoARIMA  AutoARIMA-lo-90  \
0  cluster_0 2019-08-26 2019-08-19  63.0  31.151754         9.746480   
1  cluster_0 2019-09-02 2019-08-19  15.0  31.888296         9.671510   
2  cluster_0 2019-09-09 2019-09-02  35.0  35.477863        13.414785   
3  cl

In [15]:
p.head(10)

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 17, Finished, Available, Finished)

,unique_id,ds,cutoff,y,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,cluster_0,2019-08-26,2019-08-19,63.0,31.151754,9.746480,52.557030
1,cluster_0,2019-09-02,2019-08-19,15.0,31.888296,9.671510,54.105083
2,cluster_0,2019-09-09,2019-09-02,35.0,35.477863,13.414785,57.540939
3,cluster_0,2019-09-16,2019-09-02,40.0,31.131956,8.417244,53.846668
4,cluster_0,2019-09-23,2019-09-16,42.0,36.065189,13.810037,58.320343
5,cluster_0,2019-09-30,2019-09-16,31.0,36.065189,11.852448,60.277931
6,cluster_1,2019-08-26,2019-08-19,0.0,1.190233,-1.905431,4.285896
7,cluster_1,2019-09-02,2019-08-19,4.0,0.824975,-2.323542,3.973492
8,cluster_1,2019-09-09,2019-09-02,3.0,0.983341,-2.130664,4.097346
9,cluster_1,2019-09-16,2019-09-02,3.0,1.556396,-1.607707,4.720498


In [17]:
# Add the store_id column
p['store_id'] = 'S0030'

# Reorder columns to place store_id at the beginning
p_filtered = p[['store_id'] + [col for col in p.columns if col != 'store_id']]

# Sort values by store_id and unique_id
p_filtered = p_filtered.sort_values(['store_id', 'unique_id'], ascending=True)


# Calculate evaluation metrics
p_filtered['error'] = p_filtered['y'] - p_filtered['AutoARIMA']
p_filtered['squared_error'] = (p_filtered['error']) ** 2
p_filtered['absolute_error'] = np.abs(p_filtered['error'])
p_filtered['absolute_percentage_error'] = p_filtered['absolute_error'] / p_filtered['y']

# Calculate MAE, MSE, and WMAPE at the group level
metrics_summary = p_filtered.groupby('unique_id').agg({
    'absolute_error': 'mean',  # MAE
    'squared_error': 'mean',   # MSE for RMSE calculation
    'y': 'sum',  # Sum of actual values for WMAPE
    'error': 'sum'  # Sum of absolute errors for WMAPE
}).rename(columns={
    'absolute_error': 'MAE',
    'squared_error': 'MSE',
    'y': 'sum_y',
    'error': 'sum_error'
})

# Calculate RMSE from MSE
metrics_summary['RMSE'] = np.sqrt(metrics_summary['MSE'])

# Calculate WMAPE
metrics_summary['WMAPE'] = (metrics_summary['sum_error'].abs() / metrics_summary['sum_y']) * 100

# Drop intermediate columns used for WMAPE calculation
metrics_summary = metrics_summary.drop(columns=['sum_y', 'sum_error'])

# Reset index to make unique_id a column
metrics_summary = metrics_summary.reset_index()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 19, Finished, Available, Finished)

,unique_id,MAE,MSE,RMSE,WMAPE
0,cluster_0,11.513741,239.878983,15.488027,10.716704
1,cluster_1,1.647743,3.358236,1.832549,32.688122
2,cluster_2,1.145817,1.784424,1.335823,102.708015
3,cluster_3,5.762451,60.211468,7.759605,26.062744
4,cluster_4,3.146408,13.128281,3.623297,8.648839
5,cluster_5,1.412354,2.299085,1.516273,21.462831
6,cluster_6,1.304669,2.235331,1.495102,0.311949
7,cluster_7,2.542134,7.655527,2.766862,108.605972
8,cluster_8,4.666555,41.279240,6.424892,19.278122
9,cluster_9,3.869995,29.492365,5.430687,3.687695


In [16]:
'''# Add the store_id column
p['store_id'] = 'S0030'

# Reorder columns to place store_id at the beginning
p_filtered = p[['store_id'] + [col for col in p.columns if col != 'store_id']]

# Sort values by store_id and unique_id
p_filtered = p_filtered.sort_values(['store_id', 'unique_id'], ascending=True)

# Calculate evaluation metrics
p_filtered['error'] = p_filtered['y'] - p_filtered['AutoARIMA']
p_filtered['squared_error'] = (p_filtered['error']) ** 2
p_filtered['absolute_error'] = np.abs(p_filtered['error'])
p_filtered['absolute_percentage_error'] = p_filtered['absolute_error'] / p_filtered['y']
p_filtered['weighted_absolute_percentage_error'] = p_filtered['absolute_percentage_error'] * (p_filtered['y'] / p_filtered['y'].sum())

# Aggregate metrics by unique_id
metrics_summary = p_filtered.groupby('unique_id').agg({
    'absolute_error': 'mean',  # MAE
    'squared_error': 'mean',   # MSE for RMSE calculation
    'weighted_absolute_percentage_error': 'sum'  # WMAPE
}).rename(columns={
    'absolute_error': 'MAE',
    'squared_error': 'MSE',
    'weighted_absolute_percentage_error': 'WMAPE'
})

# Calculate RMSE from MSE
metrics_summary['RMSE'] = np.sqrt(metrics_summary['MSE'])

# Drop MSE column as it's not needed in the final output
metrics_summary = metrics_summary.drop(columns='MSE')

# Reset index to make unique_id a column
metrics_summary = metrics_summary.reset_index()

# Create the final metrics DataFrame
metrics_df = pd.DataFrame(metrics_summary)


metrics_df

StatementMeta(, 975428c9-f95d-47a0-8f88-acc0eb20ed38, 18, Finished, Available, Finished)

,unique_id,MAE,WMAPE,RMSE
0,cluster_0,11.513741,0.100411,15.488027
1,cluster_1,1.647743,0.012640,1.832549
2,cluster_2,1.145817,0.004215,1.335823
3,cluster_3,5.762451,0.050254,7.759605
4,cluster_4,3.146408,0.027440,3.623297
5,cluster_5,1.412354,0.009525,1.516273
6,cluster_6,1.304669,0.011378,1.495102
7,cluster_7,2.542134,0.011231,2.766862
8,cluster_8,4.666555,0.040697,6.424892
9,cluster_9,3.869995,0.033750,5.430687


In [18]:
# save to csv
#p.to_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/predictions_noexogeneous_week_alldates_clusters_CROSSVAL_S0030.csv') 



# Salvando o DataFrame de métricas como CSV no Fabric Lakehouse
metrics_path = 'abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/METRICS_noexogeneous_week_all_dates_clusters_CROSSVAL_S0030.csv'
metrics_summary.to_csv(metrics_path, index=False)

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 20, Finished, Available, Finished)

In [70]:
'''import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Create the plot
fig, ax = plt.subplots(2, 1, figsize=(1280/96, 720/96))

# Define product IDs to plot
product_ids = ['P0015', 'P0026']

# Loop through each product ID and create a plot
for ax_, product_id in enumerate(product_ids):
    product_data = p_filtered[p_filtered['unique_id'] == product_id]
    product_data.plot(x='ds', y='y', ax=ax[ax_], label='Actual Sales', title=product_id, linewidth=2)
    product_data.plot(x='ds', y='AutoARIMA', ax=ax[ax_], label='AutoARIMA Predictions', color='orange')

    ax[ax_].set_xlabel('Week')
    ax[ax_].set_ylabel('Sales')
    ax[ax_].fill_between(
        product_data['ds'].values,
        product_data['AutoARIMA-lo-90'],
        product_data['AutoARIMA-hi-90'],
        alpha=0.2,
        color='orange'
    )
    ax[ax_].set_title(f'{product_id} - Orange band: 90% confidence interval')
    ax[ax_].legend()
    
    # Format x-axis to show weeks properly
    ax[ax_].xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
    ax[ax_].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

fig.tight_layout()
plt.show()
'''


StatementMeta(, 4f3a616f-16e4-4a27-936e-ba78afd8d72c, 72, Finished, Available, Finished)

"import matplotlib.pyplot as plt\nimport matplotlib.dates as mdates\n\n# Create the plot\nfig, ax = plt.subplots(2, 1, figsize=(1280/96, 720/96))\n\n# Define product IDs to plot\nproduct_ids = ['P0015', 'P0026']\n\n# Loop through each product ID and create a plot\nfor ax_, product_id in enumerate(product_ids):\n    product_data = p_filtered[p_filtered['unique_id'] == product_id]\n    product_data.plot(x='ds', y='y', ax=ax[ax_], label='Actual Sales', title=product_id, linewidth=2)\n    product_data.plot(x='ds', y='AutoARIMA', ax=ax[ax_], label='AutoARIMA Predictions', color='orange')\n\n    ax[ax_].set_xlabel('Week')\n    ax[ax_].set_ylabel('Sales')\n    ax[ax_].fill_between(\n        product_data['ds'].values,\n        product_data['AutoARIMA-lo-90'],\n        product_data['AutoARIMA-hi-90'],\n        alpha=0.2,\n        color='orange'\n    )\n    ax[ax_].set_title(f'{product_id} - Orange band: 90% confidence interval')\n    ax[ax_].legend()\n    \n    # Format x-axis to show weeks pro

### From 2018

2017-2018 regists not included --> choose best time intervals to evaluate metrics for ARIMA

In [19]:
# Adjust column names if needed
df_weekly = df_weekly.rename(columns={
    'cluster_id': 'unique_id',
    'date': 'ds',
    'sales': 'y'
})

#funcao do arima
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()


# Rename variable
data = df_weekly.copy()

df_product = data.copy()

# Ensure 'unique_id' and 'ds' are properly formatted
df_product['unique_id'] = df_product['unique_id'].astype(str)
df_product['ds'] = pd.to_datetime(df_product['ds'])

# Separate sales and date from exogenous variables
data2 = df_product[['ds', 'unique_id', 'y']]

# Split data in train for different time-range
train = data2.loc[data2['ds'] >= '2018-01-01']
# Count the occurrences of each product
product_counts = train['unique_id'].value_counts()
# Filter products that appear at least 5 times (to not arise error in crossvalidation)
products_to_keep = product_counts[product_counts >= 5].index
train = train[train['unique_id'].isin(products_to_keep)]

# Define AutoArima model
model = [AutoARIMA(season_length=52)]
#Forecast
sf = StatsForecast(models=model, freq='W-MON', n_jobs=-1)

#Cross-validation
crossvalidation_df = sf.cross_validation(
    df = train,
    h = 2,
    step_size = 2,
    n_windows = 3, 
    level = [90]
  )

print("crossvalidation_df:")
print(crossvalidation_df.head())
p = crossvalidation_df 
p = p.reset_index()
p.rename(columns={'index': 'unique_id'}, inplace=True)


print (p.head())

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 21, Finished, Available, Finished)

crossvalidation_df:
                  ds     cutoff     y  AutoARIMA  AutoARIMA-lo-90  \
unique_id                                                           
cluster_0 2019-08-26 2019-08-19  63.0  28.222193         8.202315   
cluster_0 2019-09-02 2019-08-19  15.0  27.496151         4.761412   
cluster_0 2019-09-09 2019-09-02  35.0  30.116714         8.752355   
cluster_0 2019-09-16 2019-09-02  40.0  29.552534         6.288080   
cluster_0 2019-09-23 2019-09-16  42.0  34.752407        13.578359   

           AutoARIMA-hi-90  
unique_id                   
cluster_0        48.242069  
cluster_0        50.230888  
cluster_0        51.481075  
cluster_0        52.816990  
cluster_0        55.926453  
   unique_id         ds     cutoff     y  AutoARIMA  AutoARIMA-lo-90  \
0  cluster_0 2019-08-26 2019-08-19  63.0  28.222193         8.202315   
1  cluster_0 2019-09-02 2019-08-19  15.0  27.496151         4.761412   
2  cluster_0 2019-09-09 2019-09-02  35.0  30.116714         8.752355   
3  cl

In [20]:
# return predictions 
p.head()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 22, Finished, Available, Finished)

,unique_id,ds,cutoff,y,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90
0,cluster_0,2019-08-26,2019-08-19,63.0,28.222193,8.202315,48.242069
1,cluster_0,2019-09-02,2019-08-19,15.0,27.496151,4.761412,50.230888
2,cluster_0,2019-09-09,2019-09-02,35.0,30.116714,8.752355,51.481075
3,cluster_0,2019-09-16,2019-09-02,40.0,29.552534,6.288080,52.816990
4,cluster_0,2019-09-23,2019-09-16,42.0,34.752407,13.578359,55.926453


In [22]:
# Add the store_id column
p['store_id'] = 'S0030'

# Reorder columns to place store_id at the beginning
p_filtered = p[['store_id'] + [col for col in p.columns if col != 'store_id']]

# Sort values by store_id and unique_id
p_filtered = p_filtered.sort_values(['store_id', 'unique_id'], ascending=True)


# Calculate evaluation metrics
p_filtered['error'] = p_filtered['y'] - p_filtered['AutoARIMA']
p_filtered['squared_error'] = (p_filtered['error']) ** 2
p_filtered['absolute_error'] = np.abs(p_filtered['error'])
p_filtered['absolute_percentage_error'] = p_filtered['absolute_error'] / p_filtered['y']

# Calculate MAE, MSE, and WMAPE at the group level
metrics_summary = p_filtered.groupby('unique_id').agg({
    'absolute_error': 'mean',  # MAE
    'squared_error': 'mean',   # MSE for RMSE calculation
    'y': 'sum',  # Sum of actual values for WMAPE
    'error': 'sum'  # Sum of absolute errors for WMAPE
}).rename(columns={
    'absolute_error': 'MAE',
    'squared_error': 'MSE',
    'y': 'sum_y',
    'error': 'sum_error'
})

# Calculate RMSE from MSE
metrics_summary['RMSE'] = np.sqrt(metrics_summary['MSE'])

# Calculate WMAPE
metrics_summary['WMAPE'] = (metrics_summary['sum_error'].abs() / metrics_summary['sum_y']) * 100

# Drop intermediate columns used for WMAPE calculation
metrics_summary = metrics_summary.drop(columns=['sum_y', 'sum_error'])

# Reset index to make unique_id a column
metrics_summary = metrics_summary.reset_index()

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 24, Finished, Available, Finished)

,unique_id,MAE,MSE,RMSE,WMAPE
0,cluster_0,12.173699,260.224823,16.131485,18.438099
1,cluster_1,1.722912,3.603344,1.898248,31.529459
2,cluster_2,0.670053,0.898377,0.947828,50.252415
3,cluster_3,5.403260,59.336407,7.703013,24.915026
4,cluster_4,2.832597,10.796813,3.285851,3.087925
5,cluster_5,1.358947,2.198547,1.482750,20.013325
6,cluster_6,1.320507,2.272748,1.507564,1.331383
7,cluster_7,1.563951,4.393539,2.096077,65.850845
8,cluster_8,5.047399,43.727036,6.612642,19.799192
9,cluster_9,3.721421,28.930716,5.378728,2.430014


In [23]:
# save to csv
#p.to_csv('abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/predictions_noexogeneous_week_from2018_clusters_CROSSVAL_S0030.csv') 



# Salvando o DataFrame de métricas como CSV no Fabric Lakehouse
metrics_path = 'abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/METRICS_noexogeneous_week_all_from2018_clusters_CROSSVAL_S0030.csv'
metrics_summary.to_csv(metrics_path, index=False)

StatementMeta(, 1f0a8896-4938-453f-8f7b-d9a80756284a, 25, Finished, Available, Finished)